In [ ]:
from typing import Optional

from tqdm import trange
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.transforms import LineGraph

import pathpyG as pp

pp.config['torch']['device'] = 'cuda'

In [ ]:
g = pp.TemporalGraph.from_edge_list([(0,1,0), (1,2,1), (1,3,1), (1,2,4), (2,3,5)])
print(g)
g = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)

In [458]:
def map_higher_order_index(edge_indices, k):
    """map node indices in k-th order edge index
    to corresponding tensor of k first-order nodes
    """ 

    # we need to reverse the node indices
    # to construct an edge_index with k-th order nodes
    
    ei = edge_indices[k].reshape(2,-1,1)
    
    j = 0
    for i in range(k-1, 0, -1):
        src_edge, tgt_edge = ei
        src = edge_indices[i][:,src_edge]
        tgt = edge_indices[i][:,tgt_edge]
        if j == 0:
            ei = torch.cat([src, tgt], dim=2)
        else:
            ei = torch.cat([src[:,:,:j], tgt], dim=2)
        j -= 1
    return ei

def lift_order_dag(dag, k):
    d = dag.data
    lg = LineGraph()
    edge_indices = {}
    edge_indices[1] = d.edge_index
    for i in range(1, k):
        d = lg(d)
        edge_indices[i+1] = d.edge_index

    # construct k-th-order edge index with dimension (m,2,k)
    return map_higher_order_index(edge_indices, k)

In [ ]:
g = pp.TemporalGraph.from_edge_list([(0,1,0), (1,2,1), (1,3,1), (1,2,4), (2,3,5)])
dag = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)
print(dag)

In [ ]:
# remap node indices and coalesce
palette, key = zip(*index_translation.items())
key = torch.tensor(key).to(pp.config['torch']['device'])
palette = torch.tensor(palette).to(pp.config['torch']['device'])

index = torch.bucketize(edge_index.ravel(), palette)
remapped = key[index].reshape(edge_index.shape)

In [460]:
lg = LineGraph()

g = pp.Graph.from_edge_list([(0,1), (1,2), (1,3), (2,4), (3,4), (4,5)])

edge_indices = {}
edge_indices[1] = g.data.edge_index
d = lg(g.data)
edge_indices[2] = d.edge_index
d = lg(d)
edge_indices[3] = d.edge_index

d = lg(d)
edge_indices[4] = d.edge_index

print('1', edge_indices[1])
print('2', edge_indices[2])
print('3', edge_indices[3])
print('4', edge_indices[4])

# first-order edge index      [[0, 1, 1, 2, 3, 4],
#                             [1, 2, 3, 4, 4, 5]]

# expected output 2nd order = [[0,1], [0,1], [1,2], [1,3], [3,4]]
#                             [[1,2], [1,3], [2,4], [3,4], [4,5]]

# expected outout 3rd-order = [[0,1,2], [0,1,3], [1,3,4], [1,2,4]]
#                             [[1,2,4], [1,3,4], [3,4,5], [2,4,5]]

# expected outout 4th-order = [[0,1,2,4], [0,1,3,4]]
#                             [[1,2,4,5], [1,3,4,5]]

1 EdgeIndex([[0, 1, 1, 2, 3, 4],
           [1, 2, 3, 4, 4, 5]], sparse_size=(5, ?), nnz=6, sort_order=row)
2 tensor([[0, 0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5, 5]])
3 tensor([[0, 1, 2, 3],
        [2, 3, 4, 5]])
4 tensor([[0, 1],
        [2, 3]])


In [464]:
map_higher_order_index(edge_indices, k=3)

2
0
1
-1


tensor([[[0, 1, 2],
         [0, 1, 3],
         [1, 2, 4],
         [1, 3, 4]],

        [[1, 2, 4],
         [1, 3, 4],
         [2, 4, 5],
         [3, 4, 5]]])

## Mapping 2nd-order edge index

In [456]:
src_2, tgt_2 = edge_indices[2].reshape(2,-1,1)
print(src_2)
print(tgt_2)

tensor([[0],
        [0],
        [1],
        [2],
        [3],
        [4]])
tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [5]])


In [457]:
# take all elements of innermost tensor in src_2 and tgt_2 and concatenate corresponding edges from lower_order edge index
src_1 = edge_indices[1][:,src_2]
tgt_1 = edge_indices[1][:,tgt_2]
print(src_1)
print(tgt_1)
ei = torch.cat([src_1, tgt_1], dim=2)
print(ei)

EdgeIndex([[[0],
            [0],
            [1],
            [1],
            [2],
            [3]],

           [[1],
            [1],
            [2],
            [3],
            [4],
            [4]]], sparse_size=(5, ?), nnz=6)
EdgeIndex([[[1],
            [1],
            [2],
            [3],
            [4],
            [4]],

           [[2],
            [3],
            [4],
            [4],
            [5],
            [5]]], sparse_size=(5, ?), nnz=6)
tensor([[[0, 1],
         [0, 1],
         [1, 2],
         [1, 3],
         [2, 4],
         [3, 4]],

        [[1, 2],
         [1, 3],
         [2, 4],
         [3, 4],
         [4, 5],
         [4, 5]]])


## Mapping third-order edge index:

In [448]:
src_3, tgt_3 = edge_indices[3].reshape(2,-1,1)
print(src_3)
print(tgt_3)

tensor([[0],
        [1],
        [2],
        [3]])
tensor([[2],
        [3],
        [4],
        [5]])


In [449]:
# map to edges in 2nd-order network
src_2 = edge_indices[2][:,src_3]
tgt_2 = edge_indices[2][:,tgt_3]
print(src_2)
print(tgt_2)
ei = torch.cat([src_2[:,:,:], tgt_2[:,:,:]], dim=2)
print(ei)

tensor([[[0],
         [0],
         [1],
         [2]],

        [[1],
         [2],
         [3],
         [4]]])
tensor([[[1],
         [2],
         [3],
         [4]],

        [[3],
         [4],
         [5],
         [5]]])
tensor([[[0, 1],
         [0, 2],
         [1, 3],
         [2, 4]],

        [[1, 3],
         [2, 4],
         [3, 5],
         [4, 5]]])


In [434]:
# map edges in 2nd-order to first-order edge sequence 
src_2, tgt_2 = ei
src_1 = edge_indices[1][:,src_2]
tgt_1 = edge_indices[1][:,tgt_2]
print(src_1)
print(tgt_1)
torch.cat([src_1[:,:,:-1], tgt_1[:,:,:]], dim=2)

EdgeIndex([[[0, 1],
            [0, 1],
            [1, 2],
            [1, 3]],

           [[1, 2],
            [1, 3],
            [2, 4],
            [3, 4]]], sparse_size=(5, ?), nnz=4)
EdgeIndex([[[1, 2],
            [1, 3],
            [2, 4],
            [3, 4]],

           [[2, 4],
            [3, 4],
            [4, 5],
            [4, 5]]], sparse_size=(5, ?), nnz=4)


tensor([[[0, 1, 2],
         [0, 1, 3],
         [1, 2, 4],
         [1, 3, 4]],

        [[1, 2, 4],
         [1, 3, 4],
         [2, 4, 5],
         [3, 4, 5]]])

## Mapping fourth-order edge-index

In [409]:
src_4, tgt_4 = edge_indices[4].reshape(2,-1,1)
print(src_4)
print(tgt_4)

tensor([[0],
        [1]])
tensor([[2],
        [3]])


In [410]:
# map to edges in 3rd-order
src_3 = edge_indices[3][:,src_4]
tgt_3 = edge_indices[3][:,tgt_4]
print(src_3)
print(tgt_3)

ei = torch.cat([src_3, tgt_3], dim=2)
print(ei)

tensor([[[0],
         [1]],

        [[2],
         [3]]])
tensor([[[2],
         [3]],

        [[4],
         [5]]])
tensor([[[0, 2],
         [1, 3]],

        [[2, 4],
         [3, 5]]])


In [390]:
# map to edges in 2nd-order
src_3, tgt_3 = ei

src_2 = edge_indices[2][:,src_3]
tgt_2 = edge_indices[2][:,tgt_3]
print(src_2)
print(tgt_2)

ei = torch.cat([src_2[:,:,:-1], tgt_2[:,:,:]], dim=2)
print(ei)

tensor([[[0, 1],
         [0, 2]],

        [[1, 3],
         [2, 4]]])
tensor([[[1, 3],
         [2, 4]],

        [[3, 5],
         [4, 5]]])
tensor([[[0, 1, 3],
         [0, 2, 4]],

        [[1, 3, 5],
         [2, 4, 5]]])


In [391]:
# map edges in 2nd-order to first-order edge sequence 
src_2, tgt_2 = ei
src_1 = edge_indices[1][:,src_2]
tgt_1 = edge_indices[1][:,tgt_2]
print(src_1)
print(tgt_1)
torch.cat([src_1[:,:,:-2], tgt_1[:,:,:]], dim=2)

EdgeIndex([[[0, 1, 2],
            [0, 1, 3]],

           [[1, 2, 4],
            [1, 3, 4]]], sparse_size=(5, ?), nnz=2)
EdgeIndex([[[1, 2, 4],
            [1, 3, 4]],

           [[2, 4, 5],
            [3, 4, 5]]], sparse_size=(5, ?), nnz=2)


tensor([[[0, 1, 2, 4],
         [0, 1, 3, 4]],

        [[1, 2, 4, 5],
         [1, 3, 4, 5]]])

In [226]:
map_higher_order_index(edge_indices, k=4)

RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [10]:
g = pp.Graph.from_edge_list([(0,1), (1,2), (1,3), (2,4), (3,4)])
lift_order_dag(g, k=2)

tensor([[[0, 1],
         [0, 1],
         [1, 2],
         [1, 3]],

        [[1, 2],
         [1, 3],
         [2, 4],
         [3, 4]]])

In [ ]:
lg = LineGraph()
ei = lg(g.data).edge_index
edge_map = g.data.edge_index
src, tgt = ei

print(edge_map)
ho_src = edge_map[:,src].t()
ho_tgt = edge_map[:,tgt].t()

ho_edge_index = torch.stack([ho_src, ho_tgt])
print(ho_edge_index)

In [ ]:
edge_map[:,1]

In [ ]:
d = Data(edge_index = ei, num_nodes=5)
ei = lg(d)
print(ei)

In [ ]:
src_edges = edges.expand(edges.size(0), -1, -1)
print(src_edges)

In [ ]:
tgt_edges = edges.expand(edges.size(0), -1, -1)
print(tgt_edges)

In [ ]:
tgt_edges[:,:,1]

In [ ]:
mask = (src_edges[:,:,1] == tgt_edges[:,:,0])
print(mask)

In [ ]:
w1 = WalkTransform(k=2)
out = w1(x, edge_index)
print('out k=2', out)

In [ ]:
# Example: 

# for each edge [i,j] add source [i,j] as many times as out-degree j 
# and dst [i,j] as many times as in-degree j

# For edge 0 -> 1 create node [0, 1] and add src [0,1] to edges twice
# For edge 1 -> 2 create node [1,2] and add tgt [1,2] to edges
# For edge 1 -> 3 create node [1,3] and add tgt [1,3] to edges
torch.cat([walks[0],walks[1]])

In [ ]:
g.data.edge_index

In [ ]:
walks = torch.tensor([[0], [0,1], [1,2], [1,3], [4], [4,5], [5,6]])

In [ ]:
node_idx = torch.tensor(g.data.node_idx).reshape(-1,1)
print(node_idx)

we have the following paths: 

0 -> 1 -> 2  
0 -> 1 -> 3  
1 -> 2 -> 3

and thus the following second-order edges: 

[0, 1] -> [1, 2]  
[0, 1] -> [1, 3]  
[1, 2] -> [2, 3]

In [ ]:
conv = DeBruijnTransform()

In [ ]:
print(node_idx)

In [ ]:
# first convolution uses first-order edges and node indices to create second-order edge index
out = conv(node_idx, g.data.edge_index)
# Problem: conv considers edges, rather than paths of length two
print(out)

In [ ]:
# second convolution uses second-order edges to create third-order edges
out = conv(out, g.data.edge_index)
print(out)